#### Day 21 - A
1 Number pad with 2 + 1 arrow pads.
Find the minimum inputs to enter the number codes in the number pad.
Get the sum of complexities for each line (length of minimum solutions * numeric portion of the input)

In [1]:
#Import Libraries and settings
from itertools import permutations, product

settings = {
    "day": 21,
    "test_data": 0
}

In [2]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    return lines

data_in = load_input(settings)

In [3]:
#How to get from one location on the arrow pad to another
#Some movements have multiple options that can lead to different length answers
arrow_moves = {
    "<":{"v":[">"], ">":[">>"], "^":[">^"], "A":[">>^"]},
    "v":{"<":["<"], "^":["^"], ">":[">"], "A":["^>", ">^"]},
    ">":{"<":["<<"], "v":["<"], "^":["<^", "^<"], "A":["^"]},
    "^":{"<":["v<"], "v":["v"], ">":["v>", ">v"], "A":[">"]},
    "A":{"<":["v<<"], "v":["<v", "v<"], ">":["v"], "^":["<"]}
}

#(x,y) coordinates of keys on the numpad
numpad_dims = {
    "7":(0,0), "8":(1,0), "9":(2,0),
    "4":(0,1), "5":(1,1), "6":(2,1),
    "1":(0,2), "2":(1,2), "3":(2,2),
               "0":(1,3), "A":(2,3)
}

In [4]:
#Derrive path(s) to go from 1 number on the num pad to another
def codify(source, dest, xy=True):
    delta_x = dest[0] - source[0]
    delta_y = dest[1] - source[1]

    if delta_x > 0:
        h_char = ">"*delta_x
    else:
        h_char = "<"*(-delta_x)

    if delta_y > 0:
        v_char = "v"*delta_y
    else:
        v_char = "^"*(-delta_y)

    #If only moving along 1 axis
    if h_char == "" or v_char == "":
        return [h_char + v_char]
    #Check if xy priority is given
    elif xy == 0:
        return [h_char + v_char, v_char + h_char]
    #X then Y
    elif xy == 1:
        return [h_char + v_char]
    #Y then X
    else:
        return [v_char + h_char]

#Get path from num to num
def path_numpad(source, dest):
    s_dim = numpad_dims[source]
    d_dim = numpad_dims[dest]

    ### Check if edge case ###

    #Going from left column to bottom row
    if (s_dim[0] == 0 and d_dim[1] == 3):
        return codify(s_dim, d_dim, xy=1)
        
    #Going from bottom row to left column
    if (s_dim[1] == 3 and d_dim[0] == 0):
        return codify(s_dim, d_dim, xy=-1)
    
    #Not an edge case
    return codify(s_dim, d_dim, xy=0)

#Build the arrow pad instructions required to enter moves on a high arrow pad
def path_arrowpad(higher, prev="A"):
    global arrow_moves
    
    lower = []
    for char in higher:
        #If the layer needs to move to a new character on the arrow pad
        if char != prev:
            res = arrow_moves[prev][char]
        #Otherwise it is already in the correct location
        else:
            res = ""

        #Add it to the lower array and add "A" between each movement
        for option in res:
            lower.append(option + "A")
        
        prev = char

    #If no moves required then just return A
    if not lower:
        return ["A"]
    return lower


In [5]:
#Set up lookup and history dictionaries
#Loop Dictionary contains the inputs required on the next level arrow pad to move from one location to another on the current arrow arrow pad
#Cost history is the length of the inputs required for an input on the current arrow pad split by n for each level of arrow pad below
def set_up_dictionaries():
    #Get every permutation of keys on the arrow pad
    keys = ["<", "v", ">", "^", "A"]
    perm = list(permutations(keys, 2))
    #This is to add permutations without moving keys
    for key in keys:
        perm.append((key, key))

    loop_dict = {}
    cost_history = {}

    #For each permutation
    for arr in perm:
        #Get the arrow_path for the next level
        after_1 = path_arrowpad(arr[1], arr[0])
        #All arrow pads start at A between each submitted input
        prev = "A"
        loop_dict[arr] = []

        #If there are multiple options, record the option with the fewest number of insturctions in the cost history dict
        for option in after_1:
            if arr not in cost_history.keys():
                cost_history[arr] = {1:len(option)}
            else:
                if len(option) < cost_history[arr][1]:
                    cost_history[arr][1] = len(option)

        #For each option, record the instructions required on the next arrow pad to implement it and add it to the loop_dict
        for option in after_1:
            option_res = []
            for char in option:
                option_res.append((prev, char))
                prev = char
            loop_dict[arr].append(option_res)

    return loop_dict, cost_history

In [6]:
#Recursive function that gets the cost of performing a instructions on the arrow pad n levels below the current one
def cost_for_n(move, n, cost_history, loop_dict):

    #If the solution is in the cost history, return that
    if n in cost_history[move].keys():
        return cost_history[move][n]
    #Otherwise derrive the value
    else:
        #Get the instructions to implement the move on the next arrow pad
        next_layer = loop_dict[move]

        #If multiple options, try all and take best
        best_sol = None

        #For each option get the cost for n-1, character by character
        for option in next_layer:
            sol_total = 0
            for part in option:
                sol_total += cost_for_n(part, n-1, cost_history, loop_dict)

            if best_sol is None:
                best_sol = sol_total
            elif sol_total < best_sol:
                best_sol = sol_total

        #Update the cost history with the derrived solution
        cost_history[move][n] = best_sol
            
        return best_sol

In [7]:
#Function to convert a desired num pad output into instructions for the top level arrow pad
def enter_num(nums):
    #Create a segement for each character in the desired output
    segments = []
    #Initialise the num pad starting at location A
    prev = "A"

    for num in nums:
        #Get the instructions required from the top level arrow pad to output each number
        segs = path_numpad(prev, num)
        segments.append(segs)
        prev = num

    #Get all segment combinations for cases where there are multiple paths
    perms = list(product(*segments))

    #Get all arrow paths for each perm
    all_paths = []
    for perm in perms:
        #Join segments with the "A" character to get the top level instructions required for this num output
        top = "A".join(perm) + "A"
        prev_char = "A"

        #Convert instructions from strings (i.e. ">^^A") to a series of moves (i.e. [("A", ">"), (">", "^")..etc])
        arrow_path = []
        for instruction in top:
            arrow_path.append((prev_char, instruction))
            prev_char = instruction

        all_paths.append(arrow_path)

    return all_paths

In [8]:
#Function to handle an inputs for a given number of robot layers
def process_input(num_input, layers, loop_dict, cost_history):
    #Convert desired output into top level arrow pad instructions
    arrow_paths = enter_num(num_input)

    #If there are multiple options, take the best one
    best_res = None
    for arrow_path in arrow_paths:
        
        #Sum the cost of all moves in this arrow path
        subtotal = 0
        for move in arrow_path:
            subtotal += cost_for_n(move, layers, cost_history, loop_dict)
        
        #Take the best result out of all arrow paths
        if best_res is None:
            best_res = subtotal
        else:
            if subtotal < best_res:
                best_res = subtotal

    return best_res

In [9]:
#Code to process all inputs

total = 0
layers = 2

#Set up the dictionaries
loop_dict, cost_history = set_up_dictionaries()

for line in data_in:
    #Get the numeric section of the input
    numeric = int(line[:-1])

    #Get the length of processing this input for the desired number of layers
    length = process_input(line, layers, loop_dict, cost_history)

    #Calculate the complexity
    complexity = numeric * length
    
    print(line, length, complexity)
    total += complexity

print("Total Complexity:", total)

140A 70 9800
170A 72 12240
169A 76 12844
803A 76 61028
129A 74 9546
Total Complexity: 105458
